In [26]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from text_processing import preprocess_text
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

df = pd.read_excel('../text_data/data.xlsx', nrows=7600, sheet_name='list_update')

In [27]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Tokenize and preprocess the text
texts = df['text'].astype(str).apply(preprocess_text).values

In [28]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
data = pad_sequences(sequences)
print(data.shape)

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


(3795, 37)


In [30]:
labels = df['id'].values
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
labels = to_categorical(labels)

with open('encoder.pickle', 'wb') as handle:
    pickle.dump(encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [31]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42, shuffle=True)

In [32]:
embedding_dim = 90
LSTM1 = 80
LSTM2 = 90
optimalizer = 'adam'
batch_size = 2
epochs = 8

In [33]:
VOCAB_SIZE = len(tokenizer.word_index) + 1
EMBEDDING_DIM = embedding_dim
MAX_LEN = len(data[0])
print(MAX_LEN)
OUTPUT_CLASSES = len(y_train[0])
print(OUTPUT_CLASSES)

37
33


Model

In [34]:
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LEN))
model.add(LSTM(LSTM1, return_sequences=True))  # Změna na True pro více vrstev LSTM
model.add(Dropout(0.5))  # Přidání Dropout vrstvy
model.add(LSTM(LSTM2))  # Další vrstva LSTM
model.add(Dropout(0.5))  # Přidání Dropout vrstvy
model.add(Dense(OUTPUT_CLASSES, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimalizer,
              metrics=['accuracy'])


In [35]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,  # Zvýšení počtu epoch
          validation_data=(X_test, y_test))

Epoch 1/8
1328/1328 [==============================] - 36s 25ms/step - loss: 3.0301 - accuracy: 0.1608 - val_loss: 2.2970 - val_accuracy: 0.3494
Epoch 2/8
1328/1328 [==============================] - 32s 24ms/step - loss: 1.7006 - accuracy: 0.4913 - val_loss: 1.4351 - val_accuracy: 0.5874
Epoch 3/8
1328/1328 [==============================] - 32s 24ms/step - loss: 0.8617 - accuracy: 0.7361 - val_loss: 1.1470 - val_accuracy: 0.6857
Epoch 4/8
1328/1328 [==============================] - 32s 24ms/step - loss: 0.5376 - accuracy: 0.8392 - val_loss: 1.1085 - val_accuracy: 0.7208
Epoch 5/8
1328/1328 [==============================] - 32s 24ms/step - loss: 0.3451 - accuracy: 0.8961 - val_loss: 1.1412 - val_accuracy: 0.7287
Epoch 6/8
1328/1328 [==============================] - 32s 24ms/step - loss: 0.2442 - accuracy: 0.9236 - val_loss: 1.0730 - val_accuracy: 0.7454
Epoch 7/8
1328/1328 [==============================] - 32s 24ms/step - loss: 0.1943 - accuracy: 0.9375 - val_loss: 1.2025 - val_ac

In [36]:
model.save('text_model_cz')

INFO:tensorflow:Assets written to: text_model_cz\assets


INFO:tensorflow:Assets written to: text_model_cz\assets


In [37]:
from keras import models
from keras.models import load_model
import pickle
from text_processing import preprocess_text
model = load_model('./text_model_cz')

In [51]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from text_processing import preprocess_text
class_names = ["lukostřelba", "baseball", "basketbal", "kulečník", "bmx", "bowling", "box", "jízda na býku", "roztleskávání", "curling", "šerm", "krasobruslení", "fotbal", "závody formule 1", "golf", "skok do výšky", "hokej", "dostihy", "hydroplánové závody", "judo", "motocyklové závody", "pole dance", "rugby", "skoky na lyžích", "snowboarding", "rychlobruslení", "surfování", "plavání", "stolní tenis", "tenis", "dráhové kolo", "volejbal", "vzpírání"]
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
with open('encoder.pickle', 'rb') as handle:
    encoder = pickle.load(handle)
    string = preprocess_text('je aktivita, kde sportovci používají luk k míření a střelbě na terči.')
sequences = tokenizer.texts_to_sequences([string])
data = pad_sequences(sequences, maxlen=37)
predictions = model.predict(data)
predicted_labels = predictions.argmax(axis=-1)
predicted_labels = encoder.inverse_transform(predicted_labels)
predicted_confidence = predictions.max(axis=-1)
print(class_names[predicted_labels[0]])
print(f"jistota: {predicted_confidence[0]*100} %")

1/1 [==============================] - 0s 65ms/step
vzpírání
jistota: 94.14280652999878 %
